In [2]:
import pandas as pd
import codecs
import nltk
from nltk.tokenize import word_tokenize
import numpy as np

In [38]:
vaccines_df = pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/vaccine_sentiment/german/df_translate_vaccine.csv", index_col=0)

In [39]:
vaccines_df.columns

Index(['url', 'original_headline', 'headline', 'date_published', 'channel',
       'newspaper', 'vaccine', 'pos_1', 'pos_2', 'original_headline_en'],
      dtype='object')

In [40]:
df_model2 = pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/vaccine_sentiment/german/df_model_2.csv", index_col=0,lineterminator='\n')

In [41]:
df_model2.columns

Index(['url', 'original_headline', 'headline', 'date_published', 'channel',
       'newspaper', 'vaccine', 'pos_1', 'pos_2'],
      dtype='object')

In [51]:
df_model2.set_index("url",inplace=True)

In [43]:
df_model2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37818 entries, 0 to 37831
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   url                37818 non-null  object
 1   original_headline  37818 non-null  object
 2   headline           37818 non-null  object
 3   date_published     37818 non-null  object
 4   channel            20758 non-null  object
 5   newspaper          37818 non-null  object
 6   vaccine            141 non-null    object
 7   pos_1              37818 non-null  object
 8   pos_2              37818 non-null  object
dtypes: object(9)
memory usage: 2.9+ MB


#### Fixing "headlines" for Die Zeit

In [44]:
zeit_covid_temp=pd.read_csv("/Users/carolinvogt/Becoming_Data_Analyst/vaccine_sentiment/german/zeit_covid_temp.csv", index_col=0)

In [45]:
zeit_covid_temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9942 entries, 0 to 10174
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   url        9942 non-null   object
 1   headline2  9942 non-null   object
dtypes: object(2)
memory usage: 233.0+ KB


In [46]:
zeit_covid_temp

,url,headline2
0,http://www.zeit.de/kultur/2020-03/gesellschaft...,ne runden tatsächlich erwachsen langweilig 2 7...
1,http://www.zeit.de/kultur/2020-03/corona-quara...,gewinnen verlierer schweigen möglichkeiten gut...
2,http://www.zeit.de/kultur/2020-03/staat-ausnah...,corona staat dennoch allzu vorsichtig wer rech...
3,http://www.zeit.de/gesellschaft/zeitgeschehen/...,kleine änderungen alltag bewirken provisorisch...
4,http://www.zeit.de/campus/2020-03/auslandsseme...,eigentlich sein gerne familie eigentlich wolle...
...,...,...
10157,http://www.zeit.de/politik/deutschland/2021-03...,trotzen astrazeneca-impfstaus lehnen bundesreg...
10159,http://www.zeit.de/wissen/gesundheit/2021-03/c...,thüringen stoppen vergabe impfterminen gesundh...
10160,http://www.zeit.de/politik/2021-03/corona-impf...,groß impfversprechen skizzieren möglich ablauf...
10165,http://www.zeit.de/wissen/gesundheit/2021-03/j...,jens spahn lässt abweichungen impfpriorisierun...


In [47]:
df_model2=df_model2.join(zeit_covid_temp.set_index('url'), on="url")

In [48]:
df_model2["headline2"].isna().sum()

28257

In [54]:
df_model2.drop_duplicates(inplace=True)

In [87]:
df_model21=df_model2[df_model2["newspaper"]=="Die Zeit"]
df_model21.drop(["headline"],axis=1, inplace=True)

/Users/carolinvogt/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [88]:
df_model21.isna().sum()

original_headline        0
date_published           0
channel                  0
newspaper                0
vaccine              11029
pos_1                    0
pos_2                    0
headline2                0
dtype: int64

In [89]:
rename_df_model21={'headline2':"headline"}

In [90]:
df_model21=df_model21.rename(columns=rename_df_model21)

In [91]:
df_model21

,original_headline,date_published,channel,newspaper,vaccine,pos_1,pos_2,headline
url,,,,,,,,
http://www.zeit.de/kultur/2020-03/gesellschaftsspiele-ruesselbande-mikado-ligretto-uno,Noch 'ne Runde! tatsächlich auch für Erwachsen...,2020-03-21,kultur,Die Zeit,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",ne runden tatsächlich erwachsen langweilig 2 7...
http://www.zeit.de/kultur/2020-03/corona-quarantaene-zeit-gesellschaftsspiele-pro-contra,Gewinnen ist was für Verlierer? mir schweigend...,2020-03-21,kultur,Die Zeit,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",gewinnen verlierer schweigen möglichkeiten gut...
http://www.zeit.de/kultur/2020-03/staat-ausnahmezustand-notstandsgesetze-coronavirus-deutschland-infektionsschutzgesetz-recht,Corona und der Staat dennoch nicht allzu vorsi...,2020-03-21,kultur,Die Zeit,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",corona staat dennoch allzu vorsichtig wer rech...
http://www.zeit.de/gesellschaft/zeitgeschehen/2020-03/wuhan-coronavirus-quarantaene-ausgangssperre-depressionen-angst-psychische-belastung,"""Kleine Änderungen im Alltag können viel bewir...",2020-03-21,gesellschaft,Die Zeit,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",kleine änderungen alltag bewirken provisorisch...
http://www.zeit.de/campus/2020-03/auslandssemester-coronavirus-italien-spanien-slowenien-studierende-ausland,"""Eigentlich wäre ich jetzt gerne bei meiner Fa...",2020-03-21,campus,Die Zeit,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",eigentlich sein gerne familie eigentlich wolle...
...,...,...,...,...,...,...,...,...
http://www.zeit.de/wissen/gesundheit/2021-03/jens-spahn-corona-impfung-strategie-priorisierung,Jens Spahn lässt Abweichungen von Impfpriorisi...,2021-03-10,wissen,Die Zeit,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",jens spahn lässt abweichungen impfpriorisierun...
http://www.zeit.de/2021/12/globale-impfstoffverteilung-coronavirus-gerechtigkeit-weltweit-impfung,Jetzt sind wir dran! Fernsehbilder aus Europa ...,2021-03-17,2021,Die Zeit,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",dran krank schwanger hundert haushalte zuständ...
http://www.zeit.de/2021/12/globale-impfstoffverteilung-coronavirus-gerechtigkeit-weltweit-impfung,Jetzt sind wir dran! Fernsehbilder aus Europa ...,2021-03-17,2021,Die Zeit,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",dran impfstoff kommen britisch hersteller astr...


In [66]:
df_model22=df_model2[df_model2["newspaper"]!="Die Zeit"]
df_model22.drop(["headline2"],axis=1,inplace=True)

/Users/carolinvogt/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [68]:
df_model22.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28247 entries, https://www.spiegel.de/panorama/italien-weitet-rote-zone-auf-das-ganze-land-aus-a-65c689c7-9a03-4e8c-9d13-78bb59fe2d44 to https://www.faz.net/premiumContent?contentId=1.6771918
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   original_headline  28247 non-null  object
 1   headline           28247 non-null  object
 2   date_published     28247 non-null  object
 3   channel            11195 non-null  object
 4   newspaper          28247 non-null  object
 5   vaccine            87 non-null     object
 6   pos_1              28247 non-null  object
 7   pos_2              28247 non-null  object
dtypes: object(8)
memory usage: 1.9+ MB


In [69]:
df_model22.isna().sum()

original_headline        0
headline                 0
date_published           0
channel              17052
newspaper                0
vaccine              28160
pos_1                    0
pos_2                    0
dtype: int64

In [73]:
df_model22

,original_headline,headline,date_published,channel,newspaper,vaccine,pos_1,pos_2
url,,,,,,,,
https://www.spiegel.de/panorama/italien-weitet-rote-zone-auf-das-ganze-land-aus-a-65c689c7-9a03-4e8c-9d13-78bb59fe2d44,coronavirus italien weitet rote zone auf das g...,coronavirus italien weiten rote zone ganze land,2020-03-09,Panorama,Der Spiegel,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch..."
https://www.spiegel.de/panorama/coronavirus-in-brandenburg-bis-zu-2250-menschen-zu-haeuslicher-quarantaene-aufgerufen-a-0dfbe9e6-f7ff-486b-bec9-f07ce7d2ca44,coronavirus in brandenburg bis zu 2250 mensche...,coronavirus brandenburg 2250 menschen häuslich...,2020-03-09,Panorama,Der Spiegel,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch..."
https://www.spiegel.de/panorama/coronavirus-bayern-will-veranstaltungen-mit-mehr-als-1000-gaesten-untersagen-a-7aef825a-7e3f-42ba-b954-14f806b926fb,wegen coronavirus bayern will veranstaltungen ...,wegen coronavirus bayern veranstaltungen mehr ...,2020-03-09,Panorama,Der Spiegel,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch..."
https://www.spiegel.de/panorama/fluechtlingshilfe-warum-mehrstetten-fuenf-fluechtlinge-zusaetzlich-aufnimmt-a-b1f2a31b-3837-43b3-a90f-459e82ccf187,dieser ort würde fünf geflüchtete mehr aufnehm...,ort fünf geflüchtete mehr aufnehmen lösung klein,2020-03-09,Panorama,Der Spiegel,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch..."
https://www.spiegel.de/politik/ausland/news-des-tages-mh17-abschuss-so-lief-der-prozessauftakt-a-9dfb685a-0846-4457-a9ca-13119ad0f353,die lage am abend so lief der prozessauftakt z...,lage abend laufen prozessauftakt mh17abschuss,2020-03-09,Politik,Der Spiegel,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch..."
...,...,...,...,...,...,...,...,...
https://www.faz.net/suche/streit-zwischen-astra-zeneca-und-eu-um-impfstoff-lieferung-17167316.html,Es knistert zwischen Astra-Zeneca und der EU ...,knistern astra-zeneca eu schwedisch-britische ...,2021-01-27,NaN,Frankfurter Allgemeine,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch..."
https://www.faz.net/premiumContent?contentId=1.7166962,Chef von Astra-Zeneca weist Vorwürfe der EU zu...,chef astra-zeneca weisen vorwürfe eu zurück eu...,2021-01-27,NaN,Frankfurter Allgemeine,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch..."
https://www.faz.net/premiumContent?contentId=1.7140631,Macron warnt vor Artensterben Auf dem K...,macron warnen artensterben klimagipfel paris g...,2021-01-11,NaN,Frankfurter Allgemeine,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch..."


In [92]:
df_model21.columns

Index(['original_headline', 'date_published', 'channel', 'newspaper',
       'vaccine', 'pos_1', 'pos_2', 'headline'],
      dtype='object')

In [93]:
df_model22.columns

Index(['original_headline', 'headline', 'date_published', 'channel',
       'newspaper', 'vaccine', 'pos_1', 'pos_2'],
      dtype='object')

In [95]:
df_model2=pd.concat([df_model21, df_model22],axis=0)
df_model2

,original_headline,date_published,channel,newspaper,vaccine,pos_1,pos_2,headline
url,,,,,,,,
http://www.zeit.de/kultur/2020-03/gesellschaftsspiele-ruesselbande-mikado-ligretto-uno,Noch 'ne Runde! tatsächlich auch für Erwachsen...,2020-03-21,kultur,Die Zeit,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",ne runden tatsächlich erwachsen langweilig 2 7...
http://www.zeit.de/kultur/2020-03/corona-quarantaene-zeit-gesellschaftsspiele-pro-contra,Gewinnen ist was für Verlierer? mir schweigend...,2020-03-21,kultur,Die Zeit,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",gewinnen verlierer schweigen möglichkeiten gut...
http://www.zeit.de/kultur/2020-03/staat-ausnahmezustand-notstandsgesetze-coronavirus-deutschland-infektionsschutzgesetz-recht,Corona und der Staat dennoch nicht allzu vorsi...,2020-03-21,kultur,Die Zeit,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",corona staat dennoch allzu vorsichtig wer rech...
http://www.zeit.de/gesellschaft/zeitgeschehen/2020-03/wuhan-coronavirus-quarantaene-ausgangssperre-depressionen-angst-psychische-belastung,"""Kleine Änderungen im Alltag können viel bewir...",2020-03-21,gesellschaft,Die Zeit,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",kleine änderungen alltag bewirken provisorisch...
http://www.zeit.de/campus/2020-03/auslandssemester-coronavirus-italien-spanien-slowenien-studierende-ausland,"""Eigentlich wäre ich jetzt gerne bei meiner Fa...",2020-03-21,campus,Die Zeit,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",eigentlich sein gerne familie eigentlich wolle...
...,...,...,...,...,...,...,...,...
https://www.faz.net/suche/streit-zwischen-astra-zeneca-und-eu-um-impfstoff-lieferung-17167316.html,Es knistert zwischen Astra-Zeneca und der EU ...,2021-01-27,NaN,Frankfurter Allgemeine,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",knistern astra-zeneca eu schwedisch-britische ...
https://www.faz.net/premiumContent?contentId=1.7166962,Chef von Astra-Zeneca weist Vorwürfe der EU zu...,2021-01-27,NaN,Frankfurter Allgemeine,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",chef astra-zeneca weisen vorwürfe eu zurück eu...
https://www.faz.net/premiumContent?contentId=1.7140631,Macron warnt vor Artensterben Auf dem K...,2021-01-11,NaN,Frankfurter Allgemeine,NaN,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",macron warnen artensterben klimagipfel paris g...


# TF-IDF


https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089

In [96]:
N = len (df_model2)

In [97]:
DF = {}
processed_text = df_model2.headline.tolist()
for i in range(len(processed_text)):
    tokens = processed_text[i]
    local = tokens.split(' ')
    for w in local:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}


In [98]:
for i in DF:
    DF[i] = len(DF[i])

In [99]:
total_vocab = [x for x in DF]

In [100]:
len(total_vocab)

66147

In [101]:
print(total_vocab)

['ne', 'runden', 'tatsächlich', 'erwachsen', 'langweilig', '2', '7', 'spieler', 'ab', '4', 'jahrensimone', 'gaul', 'krise', 'brauchen', 'bilder', 'begreifbar', 'metaphern', 'derzeitig', 'corona-krise', 'sickert', 'spätestens', 'tag', 'homeoffice', 'homeschooling', 'partie', 'spaß', 'verdienen', 'gesellschaftsspiele-tipps', 'zeit', 'online-redaktion', 'gewinnen', 'verlierer', 'schweigen', 'möglichkeiten', 'gut', 'mensch', 'okay', 'vielleicht', 'erst', 'corona', 'irgendwann', 'vorbei', 'hören', 'lieb', 'menschen', 'statt', 'klaren', 'sollen', 'hause', 'bleiben', 'gesellschaftsspiele', 'sagen', 'ach', 'zeitverschwendung', 'contra', 'pro', 'staat', 'dennoch', 'allzu', 'vorsichtig', 'wer', 'rechtzeitig', 'handeln', 'siehe', 'geraten', 'schwer', 'wasser', 'weit', 'politik', 'volksmeinung', 'entfernen', 'regierend', 'testen', 'coronakrise', 'gesetzliche', 'grenzen', 'entfesseln', 'deutsch', 'kritik', 'unabdingbar', 'kleine', 'änderungen', 'alltag', 'bewirken', 'provisorisch', 'corona-krankenh

In [102]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [106]:
from collections import Counter

doc = 0
tf_idf = {}
for i in range(N):
    headline = processed_text[i]
    new_headline = headline.split(' ')
    counter = Counter(new_headline)
    words_count = len(new_headline)
    for token in np.unique(new_headline):
        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))
        tf_idf[doc, token] = tf*idf
    doc += 1

In [107]:
tf_idf

{(0, '2'): 0.2151546828381319,
 (0, '4'): 0.22583707934729097,
 (0, '7'): 0.23421422695665453,
 (0, 'ab'): 0.10743146572822318,
 (0, 'begreifbar'): 0.3296209229876368,
 (0, 'bilder'): 0.1899657649200893,
 (0, 'brauchen'): 0.1401116228845821,
 (0, 'corona-krise'): 0.07746235589077917,
 (0, 'derzeitig'): 0.22166497458215745,
 (0, 'erwachsen'): 0.19514623504922363,
 (0, 'gaul'): 0.29907789859183165,
 (0, 'gesellschaftsspiele-tipps'): 0.3296209229876368,
 (0, 'homeoffice'): 0.1541979166908073,
 (0, 'homeschooling'): 0.21462133812658388,
 (0, 'jahrensimone'): 0.3296209229876368,
 (0, 'krise'): 0.1134918701426383,
 (0, 'langweilig'): 0.26030620493164225,
 (0, 'metaphern'): 0.30651601696897196,
 (0, 'ne'): 0.30651601696897196,
 (0, 'online-redaktion'): 0.3296209229876368,
 (0, 'partie'): 0.25457786303408697,
 (0, 'runden'): 0.22368579530937194,
 (0, 'sickert'): 0.3161054193840313,
 (0, 'spaß'): 0.2297631805358371,
 (0, 'spieler'): 0.20707756725740092,
 (0, 'spätestens'): 0.20424758579785138,


In [114]:
with open('SentiWS.txt') as f:
    contents = f.read()
    print(contents)

SentiWS
~~~~~~~

SentimentWortschatz, or SentiWS for short, is a publicly available German-language resource for sentiment analysis, opinion mining etc. It lists positive and negative polarity bearing words weighted within the interval of [-1; 1] plus their part of speech tag, and if applicable, their inflections. The current version of SentiWS (v2.0) contains around 1,650 positive and 1,800 negative words, which sum up to around 16,000 positive and around 18,000 negative word forms incl. their inflections, respectively. It not only contains adjectives and adverbs explicitly expressing a sentiment, but also nouns and verbs implicitly containing one.


License
~~~~~~~

SentiWS is licensed under a Creative Commons Attribution-Noncommercial-Share Alike 3.0 Unported License (http://creativecommons.org/licenses/by-nc-sa/3.0/).


Obtain a Copy
~~~~~~~~~~~~~
The latest version of SentiWS can be found at https://wortschatz.uni-leipzig.de/download/.


Data Format
~~~~~~~~~~~
SentiWS is organise

In [115]:
with open('SentiWS_v2.0_Negative.txt') as f:
    contents = f.read()
    print(contents)

Abbau|NN	-0.058	Abbaus,Abbaues,Abbauen,Abbaue,Abbauten
Abbruch|NN	-0.0048	Abbruches,Abbrüche,Abbruchs,Abbrüchen,Abbruche
Abdankung|NN	-0.0048	Abdankungen
Abdämpfung|NN	-0.0048	Abdämpfungen
Abfall|NN	-0.0048	Abfalles,Abfälle,Abfalls,Abfällen,Abfalle
Abfuhr|NN	-0.3367	Abfuhren
Abgrund|NN	-0.3465	Abgründe,Abgrunde,Abgrundes,Abgrunds,Abgründen
Abhängigkeit|NN	-0.3653	Abhängigkeiten
Ablehnung|NN	-0.5118	Ablehnungen
Ablenkung|NN	-0.0435	Ablenkungen
Abnahme|NN	-0.0048	Abnahmen
Abneigung|NN	-0.0048	Abneigungen
Abnutzung|NN	-0.0048	Abnutzungen
Abriss|NN	-0.0048	Abrisse,Abrissen,Abrisses
Abrutsch|NN	-0.0048	Abrutschen,Abrutsche,Abrutsches,Abrutschs
Abschaffung|NN	-0.058	Abschaffungen
Abschreckung|NN	-0.0048	Abschreckungen
Abschreibung|NN	-0.3345	Abschreibungen
Abschuß|NN	-0.0048	Abschüsse,Abschusses,Abschüssen
Abschuss|NN	-0.0048	Abschüsse,Abschusse,Abschusses,Abschüssen
Abschwächung|NN	-0.1935	Abschwächungen
Absenkung|NN	-0.3434	Absenkungen
Abspaltung|NN	-0.0048	Abspaltungen
Absperrung|NN	-0.00

Data Format

SentiWS is organised in two utf8-encoded text files structured the following way:

In [121]:
# <Word>|<POS tag> \t <Polarity weight> \t <Infl_1>,...,<Infl_k> \n

In [126]:
content_negative_test=content_negative[0:3]
content_negative_test

['Abbau|NN\t-0.058\tAbbaus,Abbaues,Abbauen,Abbaue,Abbauten\n',
 'Abbruch|NN\t-0.0048\tAbbruches,Abbrüche,Abbruchs,Abbrüchen,Abbruche\n',
 'Abdankung|NN\t-0.0048\tAbdankungen\n']

In [117]:
with open('SentiWS_v2.0_Negative.txt') as f:
    content_negative= f.readlines()

In [124]:
len(content_negative)

1827

In [118]:
content_negative

['Abbau|NN\t-0.058\tAbbaus,Abbaues,Abbauen,Abbaue,Abbauten\n',
 'Abbruch|NN\t-0.0048\tAbbruches,Abbrüche,Abbruchs,Abbrüchen,Abbruche\n',
 'Abdankung|NN\t-0.0048\tAbdankungen\n',
 'Abdämpfung|NN\t-0.0048\tAbdämpfungen\n',
 'Abfall|NN\t-0.0048\tAbfalles,Abfälle,Abfalls,Abfällen,Abfalle\n',
 'Abfuhr|NN\t-0.3367\tAbfuhren\n',
 'Abgrund|NN\t-0.3465\tAbgründe,Abgrunde,Abgrundes,Abgrunds,Abgründen\n',
 'Abhängigkeit|NN\t-0.3653\tAbhängigkeiten\n',
 'Ablehnung|NN\t-0.5118\tAblehnungen\n',
 'Ablenkung|NN\t-0.0435\tAblenkungen\n',
 'Abnahme|NN\t-0.0048\tAbnahmen\n',
 'Abneigung|NN\t-0.0048\tAbneigungen\n',
 'Abnutzung|NN\t-0.0048\tAbnutzungen\n',
 'Abriss|NN\t-0.0048\tAbrisse,Abrissen,Abrisses\n',
 'Abrutsch|NN\t-0.0048\tAbrutschen,Abrutsche,Abrutsches,Abrutschs\n',
 'Abschaffung|NN\t-0.058\tAbschaffungen\n',
 'Abschreckung|NN\t-0.0048\tAbschreckungen\n',
 'Abschreibung|NN\t-0.3345\tAbschreibungen\n',
 'Abschuß|NN\t-0.0048\tAbschüsse,Abschusses,Abschüssen\n',
 'Abschuss|NN\t-0.0048\tAbschüsse,Ab

In [141]:
word=[]
pos=[]
sentiment=[]
word_forms=[]

for line in content_negative:
    word_temp = line.split("|")
    word.append(word_temp[0])
    #print(word_temp)
    #print(word)
    
    word_temp2 = word_temp[1].split("\t")
    pos.append(word_temp2[0])
    sentiment.append(word_temp2[1])
    #print(word_temp2)
    #print(pos)
    #print(sentiment)
    
    word_temp3=word_temp2[2].split("\n")
    word_forms_append = word_temp3[0].split(",")
    word_forms.append(word_forms_append)
    #print(type(word_forms_append),word_forms_append)
    #print(word_forms)
    
    

In [143]:
df_negative=pd.DataFrame()

df_negative["word"]=[w for w in word]
df_negative["pos"]=[p for p in pos]
df_negative["sentiment"]=[s for s in sentiment]
df_negative["word_forms"]=[l for l in word_forms]

df_negative

,word,pos,sentiment,word_forms
0,Abbau,NN,-0.058,"[Abbaus, Abbaues, Abbauen, Abbaue, Abbauten]"
1,Abbruch,NN,-0.0048,"[Abbruches, Abbrüche, Abbruchs, Abbrüchen, Abb..."
2,Abdankung,NN,-0.0048,[Abdankungen]
3,Abdämpfung,NN,-0.0048,[Abdämpfungen]
4,Abfall,NN,-0.0048,"[Abfalles, Abfälle, Abfalls, Abfällen, Abfalle]"
...,...,...,...,...
1822,übersehen,VVINF,-0.5128,"[übersehe, übersiehst, übersieht, überseht, üb..."
1823,übertreiben,VVINF,-0.0372,"[übertreibe, übertreibst, übertreibt, übertrei..."
1824,übertreten,VVINF,-0.329,"[übertrete, übertrittst, übertritt, übertretet..."
1825,übertrieben,ADJX,-0.3203,"[übertriebenes, übertriebenerer, übertriebener..."


In [116]:
with open('SentiWS_v2.0_Positive.txt') as f:
    contents = f.read()
    print(contents)

Abmachung|NN	0.0040	Abmachungen
Abschluß|NN	0.0040	Abschluss,Abschlusse,Abschlusses,Abschlüsse,Abschlüssen
Abstimmung|NN	0.0040	Abstimmungen
Agilität|NN	0.0040	
Aktivität|NN	0.0040	Aktivitäten
Aktualisierung|NN	0.0040	Aktualisierungen
Aktualität|NN	0.0040	Aktualitäten
Akzeptanz|NN	0.0040	Akzeptanzen
Andrang|NN	0.0040	Andrangs,Andranges,Andrange
Anerkennung|NN	0.0855	Anerkennungen
Angebot|NN	0.0711	Angeboten,Angebote,Angebots,Angebotes
Angemessenheit|NN	0.0040	
Anhebung|NN	0.0040	Anhebungen
Anheiterung|NN	0.0040	
Ankurbelung|NN	0.0040	Ankurbelungen
Annehmlichkeit|NN	0.0040	Annehmlichkeiten
Annäherung|NN	0.0040	Annäherungen
Anpassung|NN	0.0040	Anpassungen
Anpassungsfähigkeit|NN	0.0040	Anpassungsfähigkeiten
Anreicherung|NN	0.0040	Anreicherungen
Anspruch|NN	0.0040	Anspruchs,Anspruches,Ansprüchen,Ansprüche
Anstand|NN	0.0040	Anstands,Anständen,Anstände,Anstandes
Anstieg|NN	0.0040	Anstiegen,Anstiegs,Anstiege,Anstieges
Anständigkeit|NN	0.0040	
Anteil|NN	0.0040	Anteilen,Anteiles,Anteils,Anteile

In [119]:
with open('SentiWS_v2.0_Positive.txt') as f:
    content_positive = f.readlines()

In [120]:
content_positive

['Abmachung|NN\t0.0040\tAbmachungen\n',
 'Abschluß|NN\t0.0040\tAbschluss,Abschlusse,Abschlusses,Abschlüsse,Abschlüssen\n',
 'Abstimmung|NN\t0.0040\tAbstimmungen\n',
 'Agilität|NN\t0.0040\t\n',
 'Aktivität|NN\t0.0040\tAktivitäten\n',
 'Aktualisierung|NN\t0.0040\tAktualisierungen\n',
 'Aktualität|NN\t0.0040\tAktualitäten\n',
 'Akzeptanz|NN\t0.0040\tAkzeptanzen\n',
 'Andrang|NN\t0.0040\tAndrangs,Andranges,Andrange\n',
 'Anerkennung|NN\t0.0855\tAnerkennungen\n',
 'Angebot|NN\t0.0711\tAngeboten,Angebote,Angebots,Angebotes\n',
 'Angemessenheit|NN\t0.0040\t\n',
 'Anhebung|NN\t0.0040\tAnhebungen\n',
 'Anheiterung|NN\t0.0040\t\n',
 'Ankurbelung|NN\t0.0040\tAnkurbelungen\n',
 'Annehmlichkeit|NN\t0.0040\tAnnehmlichkeiten\n',
 'Annäherung|NN\t0.0040\tAnnäherungen\n',
 'Anpassung|NN\t0.0040\tAnpassungen\n',
 'Anpassungsfähigkeit|NN\t0.0040\tAnpassungsfähigkeiten\n',
 'Anreicherung|NN\t0.0040\tAnreicherungen\n',
 'Anspruch|NN\t0.0040\tAnspruchs,Anspruches,Ansprüchen,Ansprüche\n',
 'Anstand|NN\t0.004

In [144]:
word=[]
pos=[]
sentiment=[]
word_forms=[]

for line in content_positive:
    word_temp = line.split("|")
    word.append(word_temp[0])
    #print(word_temp)
    #print(word)
    
    word_temp2 = word_temp[1].split("\t")
    pos.append(word_temp2[0])
    sentiment.append(word_temp2[1])
    #print(word_temp2)
    #print(pos)
    #print(sentiment)
    
    word_temp3=word_temp2[2].split("\n")
    word_forms_append = word_temp3[0].split(",")
    word_forms.append(word_forms_append)
    #print(type(word_forms_append),word_forms_append)
    #print(word_forms)
    
    

In [145]:
df_positive=pd.DataFrame()

df_positive["word"]=[w for w in word]
df_positive["pos"]=[p for p in pos]
df_positive["sentiment"]=[s for s in sentiment]
df_positive["word_forms"]=[l for l in word_forms]

df_positive

,word,pos,sentiment,word_forms
0,Abmachung,NN,0.0040,[Abmachungen]
1,Abschluß,NN,0.0040,"[Abschluss, Abschlusse, Abschlusses, Abschlüss..."
2,Abstimmung,NN,0.0040,[Abstimmungen]
3,Agilität,NN,0.0040,[]
4,Aktivität,NN,0.0040,[Aktivitäten]
...,...,...,...,...
1639,überwältigt,ADJX,0.0578,"[überwältigter, überwältigtes, überwältigte, ü..."
1640,überzeugen,VVINF,0.3396,"[überzeuge, überzeugst, überzeugt, überzeuget,..."
1641,überzeugend,ADJX,0.4489,"[überzeugender, überzeugendes, überzeugende, ü..."
1642,überzeugt,ADJX,0.2381,"[überzeugtestem, überzeugtesten, überzeugteste..."


In [148]:
df_senitment=pd.concat([df_positive, df_negative], axis=0)

In [168]:
df_senitment.reset_index(inplace=True)

In [169]:
index=list(df_senitment.index)
#index

In [170]:
df_senitment["sentiment"][0]

'0.0040'

In [179]:
type(df_senitment["word_forms"][0])

list

In [182]:
df_senitment["word_forms"][0]

['Abmachungen']

In [183]:
sentiment_dic={}

for i in index:
    sentiment_dic[df_senitment["word"][i].lower()]=(df_senitment["sentiment"][i])
    for word in df_senitment["word_forms"][i]:
        if len(word)>1:
            sentiment_dic[word.lower()]=(df_senitment["sentiment"][i])

In [184]:
sentiment_dic

{'abmachung': '0.0040',
 'abmachungen': '0.0040',
 'abschluß': '0.0040',
 'abschluss': '0.0040',
 'abschlusse': '0.0040',
 'abschlusses': '0.0040',
 'abschlüsse': '0.0040',
 'abschlüssen': '0.0040',
 'abstimmung': '0.0040',
 'abstimmungen': '0.0040',
 'agilität': '0.0040',
 'aktivität': '0.0040',
 'aktivitäten': '0.0040',
 'aktualisierung': '0.0040',
 'aktualisierungen': '0.0040',
 'aktualität': '0.0040',
 'aktualitäten': '0.0040',
 'akzeptanz': '0.0040',
 'akzeptanzen': '0.0040',
 'andrang': '0.0040',
 'andrangs': '0.0040',
 'andranges': '0.0040',
 'andrange': '0.0040',
 'anerkennung': '0.0855',
 'anerkennungen': '0.0855',
 'angebot': '0.0711',
 'angeboten': '0.0711',
 'angebote': '0.0711',
 'angebots': '0.0711',
 'angebotes': '0.0711',
 'angemessenheit': '0.0040',
 'anhebung': '0.0040',
 'anhebungen': '0.0040',
 'anheiterung': '0.0040',
 'ankurbelung': '0.0040',
 'ankurbelungen': '0.0040',
 'annehmlichkeit': '0.0040',
 'annehmlichkeiten': '0.0040',
 'annäherung': '0.0040',
 'annäheru

In [153]:
tf_idf[(0, 'begreifbar')]

0.3296209229876368

In [185]:
doc = 0
sentiment = {}
for i in range(N):
    headline = processed_text[i]
    new_headline = headline.split(' ')
    for token in np.unique(new_headline):
        try:
            sentiment_value=sentiment_dic[token]
            sentiment[doc, token] = sentiment_value
        except:
            sentiment[doc, token] = 0
    doc += 1

In [186]:
sentiment

{(0, '2'): 0,
 (0, '4'): 0,
 (0, '7'): 0,
 (0, 'ab'): 0,
 (0, 'begreifbar'): 0,
 (0, 'bilder'): 0,
 (0, 'brauchen'): 0,
 (0, 'corona-krise'): 0,
 (0, 'derzeitig'): 0,
 (0, 'erwachsen'): 0,
 (0, 'gaul'): 0,
 (0, 'gesellschaftsspiele-tipps'): 0,
 (0, 'homeoffice'): 0,
 (0, 'homeschooling'): 0,
 (0, 'jahrensimone'): 0,
 (0, 'krise'): '-0.3621',
 (0, 'langweilig'): '-0.0228',
 (0, 'metaphern'): 0,
 (0, 'ne'): 0,
 (0, 'online-redaktion'): 0,
 (0, 'partie'): 0,
 (0, 'runden'): 0,
 (0, 'sickert'): 0,
 (0, 'spaß'): '0.2823',
 (0, 'spieler'): 0,
 (0, 'spätestens'): 0,
 (0, 'tag'): 0,
 (0, 'tatsächlich'): 0,
 (0, 'verdienen'): '0.0989',
 (0, 'zeit'): 0,
 (1, 'ach'): 0,
 (1, 'bleiben'): 0,
 (1, 'contra'): 0,
 (1, 'corona'): 0,
 (1, 'erst'): 0,
 (1, 'gesellschaftsspiele'): 0,
 (1, 'gewinnen'): '0.0040',
 (1, 'gut'): '0.3716',
 (1, 'hause'): 0,
 (1, 'hören'): 0,
 (1, 'irgendwann'): 0,
 (1, 'klaren'): '0.0040',
 (1, 'lieb'): '0.1131',
 (1, 'mensch'): 0,
 (1, 'menschen'): 0,
 (1, 'möglichkeiten'): '0

In [ ]:
sentiment

In [189]:
test=df_model2.reset_index(drop=False, inplace=False)
df_model2=test

In [192]:
index_df=list(df_model2.index)
#index_df

In [ ]:
if (3,"bewirken") in list(sum_sentiment.keys())

In [214]:
N = len (df_model2)

sum_sentiment={}

for index in index_df:
    headline = processed_text[index]
    new_headline = headline.split(' ')
    #print(new_headline)
    for token in np.unique(new_headline):
        #print(index)
        #print(token)
        if index in list(sum_sentiment.keys()):
            sentiment_value=float(sentiment[(index, token)])*float(tf_idf[(index, token)])
            sum_sentiment[index] += sentiment_value
        else:
            sum_sentiment[index]=float(sentiment[(index, token)])*float(tf_idf[(index, token)])
            
#sentiment[20, token] 

In [216]:
sum_sentiment

{0: 0.0352167535271333,
 1: -0.13719321221854122,
 2: -0.026576222888013062,
 3: -0.0005095617832267586,
 4: 0.0,
 5: -0.19123207243464524,
 6: 0.0005637429057990347,
 7: 0.017772274750934187,
 8: -0.2594296542662269,
 9: 0.033320214959327334,
 10: -0.026278715329508474,
 11: 0.0,
 12: -0.07544014815288264,
 13: -0.218917795020169,
 14: 0.05696557941243205,
 15: 0.07569149664999696,
 16: -0.05529183275500437,
 17: -0.04005344571867076,
 18: 0.04591481983031985,
 19: 0.05013153670173451,
 20: -0.04797973041129162,
 21: -0.08570020959181725,
 22: 0.07345083358824239,
 23: -0.19663552761933273,
 24: 0.0,
 25: -0.06134869310282906,
 26: -0.010517303289523265,
 27: 0.11427943557602702,
 28: -0.11880159740858524,
 29: -0.06550494162717846,
 30: 0.0014869787045254693,
 31: -0.049547144600409154,
 32: 0.03313070089905108,
 33: -0.07923977337051402,
 34: -0.14341250559637686,
 35: 0.034728393047270795,
 36: -0.02635302231778313,
 37: -0.0875117098867985,
 38: -0.17056914323156797,
 39: -0.02871

In [221]:
df_model2["sentiment"]=df_model2.apply(lambda row: sum_sentiment[row.name],axis=1)

In [224]:
df_model2[df_model2["vaccine"]=="biontech"]

,url,original_headline,date_published,channel,newspaper,vaccine,pos_1,pos_2,headline,sentiment
4891,http://www.zeit.de/wirtschaft/unternehmen/2020...,BioNTech und Pfizer starten Corona-Impfstoffst...,2020-09-07,wirtschaft,Die Zeit,biontech,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",biontech pfizer starten corona-impfstoffstudie...,-0.040371
4892,http://www.zeit.de/wirtschaft/unternehmen/2020...,BioNTech und Pfizer starten Corona-Impfstoffst...,2020-09-07,wirtschaft,Die Zeit,biontech,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",biontech pfizer starten corona-impfstoffstudie...,0.001345
4893,http://www.zeit.de/wirtschaft/unternehmen/2020...,BioNTech und Pfizer starten Corona-Impfstoffst...,2020-09-07,wirtschaft,Die Zeit,biontech,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",biontech pfizer starten corona-impfstoffstudie...,0.016511
4957,http://www.zeit.de/wirtschaft/unternehmen/2020...,BioNTech will Werk für mögliche Impfstoffprodu...,2020-09-17,wirtschaft,Die Zeit,biontech,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",biontech werk möglich impfstoffproduktion über...,0.055996
6315,http://www.zeit.de/wissen/gesundheit/2020-11/p...,BioNTech will USA bei Corona-Impfstoff nicht b...,2020-11-11,wissen,Die Zeit,biontech,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",biontech usa corona-impfstoff bevorzugen mainz...,0.051306
6365,http://www.zeit.de/wirtschaft/unternehmen/2020...,BioNTech meldet 90-prozentige Wirksamkeit von ...,2020-11-09,wirtschaft,Die Zeit,biontech,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",biontech melden 90-prozentige wirksamkeit impf...,-0.124599
6366,http://www.zeit.de/wirtschaft/unternehmen/2020...,BioNTech meldet 90-prozentige Wirksamkeit von ...,2020-11-09,wirtschaft,Die Zeit,biontech,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",biontech melden 90-prozentige wirksamkeit impf...,-0.123793
6417,http://www.zeit.de/wissen/gesundheit/2020-11/b...,BioNTech und Pfizer beantragen US-Notfallzulas...,2020-11-20,wissen,Die Zeit,biontech,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",biontech pfizer beantragen us-notfallzulassung...,0.010034
6418,http://www.zeit.de/wissen/gesundheit/2020-11/b...,BioNTech und Pfizer beantragen US-Notfallzulas...,2020-11-20,wissen,Die Zeit,biontech,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",biontech pfizer beantragen us-notfallzulassung...,-0.008777
8361,http://www.zeit.de/wissen/gesundheit/2021-02/c...,BioNTech will mehr Impfstoff an die EU liefern...,2021-02-01,wissen,Die Zeit,biontech,"[['Noch', 'ADV'], [""'ne"", 'ART'], ['Runde', 'N...","[['Noch', 'ADV'], ['Runde', 'NOUN'], ['tatsäch...",biontech mehr impfstoff eu liefern biontech fr...,0.002536


# Word Sense Disambiguation

Will be done for the englisch translation of german headlines. Not applicable for german language. Possibilities:


OdeNet (OpenSource): https://github.com/hdaSprachtechnologie/odenet

GermaNet (Licence required): https://uni-tuebingen.de/en/faculties/faculty-of-humanities/departments/modern-languages/department-of-linguistics/chairs/general-and-computational-linguistics/ressources/lexica/germanet/

In [14]:
from nltk.wsd import lesk

x = 'The porn threatens to cause a lockdown'

print(lesk(x.split(), 'threatens', 'v'))


Synset('threaten.v.03')


In [15]:
def word_definitions(title, lemmed_title):
    synsets = []
    original_text = title
    lemmed_pos = lemmed_title
    for i in lemmed_pos:
        synsets.append(lesk(original_text.split(), i[0], i[1]))
    return synsets

In [21]:
x = newspapers['title'][1]
y = newspapers['lem_pos_headline'][1]
import re
from nltk.corpus import sentiwordnet as swn,SentiSynset

j = word_definitions(x, y)
k = str(j[0])
print(k)
x = (re.findall(r"'([^']*)'", k))
print(x)
f = swn.senti_synset(x[0])
print(f)

Synset('agnosticism.n.02')
['agnosticism.n.02']
<agnosticism.n.02: PosScore=0.25 NegScore=0.0>


In [19]:
newspapers['word_definitions'] = newspapers.apply(lambda x: word_definitions(x['title'], x['lem_pos_headline']), axis=1)


In [20]:
newspapers

,hrefs,title,new_title,id,date,newspaper_name,vaccine,pos_headline,lem_pos_headline,word_definitions
0,https://www.theguardian.com/politics/2021/mar/...,Boris Johnson receives Oxford/AstraZeneca Covi...,bor johnson receiv oxfordastrazenec covid vaccin,politics,2021-03-19,guardian,NaN,"[[(Boris, NNP)], [(Johnson, NNP)], [(receives,...","[[receives, n], [Covid, n], [vaccine, n]]","[None, None, Synset('vaccine.n.01')]"
1,https://www.theguardian.com/world/2021/feb/19/...,Scepticism over Oxford vaccine threatens Europ...,sceiv oxford vaccin threatens europ immun push,world,2021-02-19,guardian,NaN,"[[(Scepticism, NN)], [(over, IN)], [(Oxford, N...","[[Scepticism, n], [Oxford, n], [vaccine, n], [...","[Synset('agnosticism.n.02'), Synset('oxford.n...."
2,https://www.theguardian.com/world/2021/mar/18/...,Thursday briefing: EU's experts to give Oxford...,thursday brief eu expert giv oxford vaccin ver...,world,2021-03-18,guardian,NaN,"[[(Thursday, NNP)], [(experts, NNS)], [(to, TO...","[[expert, n], [give, v], [Oxford, n], [vaccine...","[Synset('expert.n.01'), Synset('give.v.18'), S..."
3,https://www.theguardian.com/world/2021/mar/16/...,Oxford/AstraZeneca vaccine: which countries ha...,oxfordastrazenec vaccin country paus jab,world,2021-03-16,guardian,NaN,"[[(which, WDT)], [(countries, NNS)], [(have, V...","[[country, n], [have, v], [pause, v], [jab, n]]","[Synset('country.n.04'), Synset('take.v.35'), ..."
4,https://www.theguardian.com/world/2021/mar/16/...,Chaos in Germany and Italy after suspension of...,chao germany ita suspend oxford vaccin,world,2021-03-16,guardian,NaN,"[[(Chaos, NN)], [(in, IN)], [(Germany, NNP)], ...","[[Chaos, n], [suspension, n], [Oxford, n], [va...","[Synset('chaos.n.02'), Synset('suspension.n.05..."
...,...,...,...,...,...,...,...,...,...,...
62019,https://www.independent.co.uk/news/uk/politics...,Extreme lockdown laws extended for a further s...,extrem lockdown law extend six month despit ma...,coronavirus,2021-03-25,independent,NaN,"[[(Extreme, NN)], [(lockdown, NN)], [(laws, NN...","[[Extreme, n], [lockdown, n], [law, n], [exten...","[Synset('extreme_point.n.01'), Synset('lockdow..."
62020,https://www.independent.co.uk/news/uk/politics...,Extreme lockdown laws extended for a further s...,extrem lockdown law extend six month despit ma...,coronavirus,2021-03-25,independent,NaN,"[[(Extreme, NN)], [(lockdown, NN)], [(laws, NN...","[[Extreme, n], [lockdown, n], [law, n], [exten...","[Synset('extreme_point.n.01'), Synset('lockdow..."
62021,https://www.independent.co.uk/independentpremi...,The problem with Johnson’s vaccine passports f...,the problem johnson vaccin passport pub,business,2021-03-25,independent,NaN,"[[(The, DT)], [(problem, NN)], [(with, IN)], [...","[[problem, n], [vaccine, n], [passport, n], [p...","[Synset('problem.n.02'), Synset('vaccine.n.01'..."
62022,https://www.independent.co.uk/voices/editorial...,Reality is dawning on the EU: export bans are ...,real dawn eu export ban counterproduc cas covi...,uk,2021-03-25,independent,NaN,"[[(Reality, NN)], [(is, VBZ)], [(dawning, VBG)...","[[Reality, n], [be, v], [dawn, v], [export, n]...","[Synset('reality.n.03'), Synset('embody.v.02')..."


In [ ]:
tf_idf[(0, 'bor')]